In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import matplotlib.pyplot as plt

tf.enable_eager_execution()
tf.__version__

'1.14.0'

# 당뇨병을 진단해봅시다.

In [2]:
xy = np.loadtxt('./data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[0:-101, 0:-1]
y_train = xy[0:-101, [-1]]

x_test = xy[-101:, 0:-1]
y_test = xy[-101:, [-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(658, 8) (658, 1)
(101, 8) (101, 1)


In [3]:
print(x_test)
print(y_test)

[[ 0.176471    0.115578    0.147541   -0.454545    0.         -0.180328
  -0.9462     -0.366667  ]
 [-0.294118   -0.0150754  -0.0491803  -0.333333   -0.550827    0.0134128
  -0.699402   -0.266667  ]
 [ 0.0588235   0.547739    0.278689   -0.393939   -0.763593   -0.0789866
  -0.926558   -0.2       ]
 [-0.294118    0.658291    0.114754   -0.474747   -0.602837    0.00149028
  -0.527754   -0.0666667 ]
 [-0.882353   -0.00502513 -0.0491803  -0.79798     0.         -0.242921
  -0.596072    0.        ]
 [ 0.176471   -0.316583    0.737705   -0.535354   -0.884161    0.0581222
  -0.823228   -0.133333  ]
 [-0.647059    0.236181    0.639344   -0.292929   -0.432624    0.707899
  -0.315115   -0.966667  ]
 [-0.0588235  -0.0854271   0.344262    0.          0.          0.0611028
  -0.565329    0.566667  ]
 [-0.294118    0.959799    0.147541    0.          0.         -0.0789866
  -0.786507   -0.666667  ]
 [ 0.0588235   0.567839    0.409836    0.          0.         -0.260805
  -0.870196    0.0666667 ]
 [ 

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


In [5]:
W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')


## 가설 설정
* 병이 있다 / 없다로 분류
* softmax classification으로 진행

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [6]:
def logistic_regression(features):
    hypothesis  = tf.div(1., 1. + tf.exp(-(tf.matmul(features, W) + b)))
    return hypothesis
  
print(logistic_regression(x_train))

W0724 15:39:04.954103 139708987004736 deprecation.py:323] From <ipython-input-6-e2e5b5b71668>:2: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


tf.Tensor(
[[0.37773952]
 [0.13443772]
 [0.46564007]
 [0.2916075 ]
 [0.74885154]
 [0.1559334 ]
 [0.18513007]
 [0.15704654]
 [0.23781577]
 [0.24836878]
 [0.09097096]
 [0.32200155]
 [0.6716114 ]
 [0.23553291]
 [0.17886318]
 [0.21916026]
 [0.15931898]
 [0.11902647]
 [0.26689228]
 [0.21575487]
 [0.20142308]
 [0.31631815]
 [0.26242572]
 [0.5052946 ]
 [0.19394264]
 [0.32037237]
 [0.7987788 ]
 [0.1539694 ]
 [0.31168002]
 [0.3916409 ]
 [0.33834013]
 [0.16467041]
 [0.44247392]
 [0.440397  ]
 [0.299425  ]
 [0.33208215]
 [0.08662476]
 [0.4648785 ]
 [0.4134352 ]
 [0.22162321]
 [0.39432198]
 [0.60121495]
 [0.29520854]
 [0.6116822 ]
 [0.16232567]
 [0.13729545]
 [0.16297765]
 [0.1261273 ]
 [0.42282304]
 [0.38975102]
 [0.41001707]
 [0.5678722 ]
 [0.38667345]
 [0.22928685]
 [0.36778668]
 [0.25071806]
 [0.55306876]
 [0.1835083 ]
 [0.10305032]
 [0.21851334]
 [0.12850824]
 [0.3932526 ]
 [0.17812863]
 [0.18447009]
 [0.38827285]
 [0.16360828]
 [0.3839589 ]
 [0.37231082]
 [0.37382734]
 [0.36309963]
 [0.29781

## Loss Function

* 기존 MSE 대신 Cross Entropy 사용

## $$
\begin{align}
loss(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [7]:
def loss_fn(hypothesis, labels):
    loss = -tf.reduce_mean(labels * tf.log(hypothesis) + \
                           (1 - labels) * tf.log(1 - hypothesis))
    return loss

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)

## 학습

In [8]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(logistic_regression(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

Iter: 0, Loss: 1.0636
Iter: 100, Loss: 0.9294
Iter: 200, Loss: 0.8424
Iter: 300, Loss: 0.7867
Iter: 400, Loss: 0.7508
Iter: 500, Loss: 0.7270
Iter: 600, Loss: 0.7106
Iter: 700, Loss: 0.6987
Iter: 800, Loss: 0.6896
Iter: 900, Loss: 0.6823
Iter: 1000, Loss: 0.6761
Iter: 1100, Loss: 0.6706
Iter: 1200, Loss: 0.6656
Iter: 1300, Loss: 0.6610
Iter: 1400, Loss: 0.6567
Iter: 1500, Loss: 0.6526
Iter: 1600, Loss: 0.6486
Iter: 1700, Loss: 0.6447
Iter: 1800, Loss: 0.6410
Iter: 1900, Loss: 0.6373
Iter: 2000, Loss: 0.6338
Iter: 2100, Loss: 0.6303
Iter: 2200, Loss: 0.6270
Iter: 2300, Loss: 0.6237
Iter: 2400, Loss: 0.6205
Iter: 2500, Loss: 0.6174
Iter: 2600, Loss: 0.6143
Iter: 2700, Loss: 0.6113
Iter: 2800, Loss: 0.6084
Iter: 2900, Loss: 0.6056
Iter: 3000, Loss: 0.6028
Iter: 3100, Loss: 0.6001
Iter: 3200, Loss: 0.5975
Iter: 3300, Loss: 0.5949
Iter: 3400, Loss: 0.5924
Iter: 3500, Loss: 0.5899
Iter: 3600, Loss: 0.5875
Iter: 3700, Loss: 0.5852
Iter: 3800, Loss: 0.5829
Iter: 3900, Loss: 0.5807
Iter: 4000, 

## 테스트

In [9]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [10]:
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.7129
